## Opentender.eu

Open Data estratti attraverso crawling da sorgenti nazionali. Per maggiori info circa il progetto e come sono stati estratti i dati, vai [qui](http://digiwhist.eu/wp-content/uploads/2016/06/D2.4-final.pdf). 

Il download del dataset può avvenire in 2 modalità distinte:

- https://opentender.eu/it/download (91756 data record)
- https://opentender.eu/all/region/IT (download possibile da interfaccia tramite button, 33199 data record). Conviene scaricare i dati in formato json per non avere problemi alla lettura del dataset a causa dei separatori.


In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_json("data/open_tender/29b5a17b771e70eaa61ea7ce9ee1fb1c.tenders.json")
#df.info()

In [3]:
def getMainCPV(x):
    try:
        if(x == np.nan):
            return np.NaN
        elif(len(x)>0):
            for el in x:
                if el["isMain"] == True:
                    #print(el)
                    return el["code"]
        else:
            return np.NaN
    except:
        return np.NaN
def filterIT(s):
    try:
        m_obj = re.search(r'^(302|35710000|39134000|39134100|42960000|48|503|72|80533)', s)
        if m_obj:
            return True
        return False
    
    except:
        return False
    
def getfinalPriceANDpublicationDate(x):
     try:
            price = x["netAmountEur"]
            date = x["publicationDate"]
            return (price, date)
    
     except:
            return (np.NaN, np.NaN)
        
def getMainBuyerName(x):
    try:
        if(x == np.nan):
            return np.NaN
        elif(len(x)>0):
                return x[0]["name"]
        else:
            return np.NaN
    except:
        return np.NaN

In [4]:
df["main_cpv"] = df.cpvs.apply(getMainCPV)
it_df = df[df.main_cpv.apply(filterIT)]

In [5]:
it_df.columns

Index(['addressOfImplementation', 'administrators', 'appealBodyName',
       'areVariantsAccepted', 'awardCriteria', 'awardDecisionDate',
       'bidDeadline', 'buyers', 'corrections', 'country', 'cpvs', 'created',
       'deposits', 'description', 'documentsDeadline', 'documentsPayable',
       'documentsPrice', 'economicRequirements', 'eligibleBidLanguages',
       'envisagedMinCandidatesCount', 'estimatedCompletionDate',
       'estimatedDurationInDays', 'estimatedDurationInMonths',
       'estimatedPrice', 'estimatedStartDate', 'finalPrice', 'fundings',
       'hasLots', 'hasOptions', 'id', 'indicators', 'isAcceleratedProcedure',
       'isCentralProcurement', 'isCoveredByGpa', 'isDocumentsAccessRestricted',
       'isDps', 'isEInvoiceAccepted', 'isElectronicAuction',
       'isFrameworkAgreement', 'isJointProcurement', 'isOnBehalfOf',
       'isWholeTenderCancelled', 'lots', 'maxBidsCount',
       'maxFrameworkAgreementParticipants', 'mediationBodyName', 'modified',
       'nation

In [6]:
it_df['bidDeadline'] = pd.to_datetime(it_df['bidDeadline'], errors='coerce')
it_df["buyer_name"] = it_df.buyers.apply(getMainBuyerName)

/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Missing Values

Di seguito è riportato per ogni campo la percentuale di missing values. 

In [7]:
#Missing values

num = it_df.isnull().sum()
den = len(it_df)
a = round(num/den, 2)
a.sort_values()

id                                   0.00
supplyType                           0.00
publications                         0.00
ot                                   0.00
modified                             0.00
isFrameworkAgreement                 0.00
isElectronicAuction                  0.00
isDps                                0.00
isCoveredByGpa                       0.00
indicators                           0.00
main_cpv                             0.00
buyer_name                           0.00
buyers                               0.00
created                              0.00
cpvs                                 0.00
title                                0.04
procedureType                        0.04
country                              0.04
description                          0.04
titleEnglish                         0.04
nationalProcedureType                0.08
selectionMethod                      0.13
lots                                 0.15
isOnBehalfOf                      

In [8]:
pricePublication = list(zip(*it_df.finalPrice.apply(getfinalPriceANDpublicationDate)))
it_df["price"] = pricePublication[0]
it_df["publicationDate"] = pricePublication[1]

/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
pd.options.display.max_colwidth = 100
it_df[~it_df.publications.isnull()][["publications", "finalPrice", "bidDeadline","title"]].head()

,publications,finalPrice,bidDeadline,title
40,"[{'source': 'http://ted.europa.eu', 'sourceId': '2011/S 121-201027', 'publicationDate': '2011-06...","{'netAmount': 11154270, 'currency': 'EUR', 'netAmountEur': 11154270, 'currencyNational': 'EUR', ...",2011-09-06 12:00:00,Procedura aperta per l’affidamento dei servizi SOC - Services Operation Center. CIG n. 269882767D.
67,"[{'source': 'http://ted.europa.eu', 'sourceId': '2012/S 183-300238', 'humanReadableUrl': 'http:/...",NaN,NaT,Servizi di system integration per la realizzazione del sistema BMS Plus.
87,"[{'source': 'http://ted.europa.eu', 'sourceId': '2015/S 50-086961', 'humanReadableUrl': 'http://...",NaN,NaT,Fornitura di licenze d'uso software e di relativi servizi di manutenzione nell'ambito del proget...
115,"[{'source': 'http://ted.europa.eu', 'sourceId': '2015/S 52-090764', 'buyerAssignedId': 'RDA n. 1...","{'netAmount': 761085, 'currency': 'EUR', 'netAmountEur': 761085, 'currencyNational': 'EUR', 'net...",2015-04-28 12:00:00,Procedura aperta per l'affidamento dei «Servizi di supporto tecnico inerenti la lavorazione dei ...
159,"[{'source': 'http://ted.europa.eu', 'sourceId': '2011/S 151-250602', 'humanReadableUrl': 'http:/...","{'netAmount': 2250000, 'currency': 'EUR', 'netAmountEur': 2250000, 'currencyNational': 'EUR', 'n...",NaT,"Procedura ristretta volta alla gestione, sviluppo e manutenzione del sistema informativo del bil..."


## Analisi esplorativa

### Distribuzione appalti per anno
Si osserva di seguito la distribuzione degli appalti per anno.
Si osserva un maggior numero di osservazioni per gli anni 2015, 2016, 2017.

### Pulizia dei dati
Si osserva che di circa 2000 appalti, 1090 non hanno un costo associato. Inoltre alcuni hanno valori improbabili (e.g. 2 data record hanno un prezzo di appalto < 100 euro).


In [10]:
it_df["bidDeadlineYear"] = it_df.bidDeadline.apply(lambda x: x.year)
it_df["bidDeadlineYear"].value_counts()

/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2016.0    250
2017.0    244
2015.0    203
2014.0    181
2013.0    163
2012.0    138
2011.0    110
2008.0      4
2010.0      4
2006.0      3
2009.0      2
2007.0      2
2018.0      1
2022.0      1
Name: bidDeadlineYear, dtype: int64

In [11]:
#numero di data record con price nullo
it_df[it_df["price"].isnull()].shape

(1090, 63)

In [12]:
#data record con price < 100

pd.options.display.max_colwidth = 100
it_df[it_df["price"] < 100]

,addressOfImplementation,administrators,appealBodyName,areVariantsAccepted,awardCriteria,awardDecisionDate,bidDeadline,buyers,corrections,country,...,selectionMethod,supplyType,technicalRequirements,title,titleEnglish,main_cpv,buyer_name,price,publicationDate,bidDeadlineYear
9983,NaN,NaN,TAR dell'Umbria Via Baglioni Perugia 06121 +39 0755755311 +39 07557322548,NaN,NaN,NaN,NaT,"[{'name': 'Comune di Perugia — S.O. Contratti e Semplificazione — Vicesegretario', 'address': {'...",NaN,IT,...,MEAT,SERVICES,NaN,Procedura aperta per l'affidamento del servizio di gestione degli atti sanzionatori della Polizi...,Data management services,72322000,Comune di Perugia — S.O. Contratti e Semplificazione — Vicesegretario,1.83,2017-05-26,NaN
10848,NaN,NaN,TAR Campania,0.0,"[{'name': 'Come da disciplinare', 'weight': 100}]",NaN,2014-01-28 12:00:00,[{'name': 'Ministero delle Infrastrutture e dei Trasporti Provveditorato Interregionale alle OO....,NaN,IT,...,MEAT,SERVICES,1. Certificazione di sistema di qualità di cui all'art. 43 del D. Lgs. 163/2006 e s.m.i. conform...,Affidamento in concesione del servizio di gestione del procedimento sanzionatorio delle violazio...,"IT services: consulting, software development, Internet and support",72000000,Ministero delle Infrastrutture e dei Trasporti Provveditorato Interregionale alle OO.PP. Campani...,16.50,2014-06-17,2014.0


In [13]:
pd.options.display.max_colwidth = 1000
it_df[["buyers", "buyer_name","bidDeadline", "finalPrice", "publicationDate", "price", "title"]]

,buyers,buyer_name,bidDeadline,finalPrice,publicationDate,price,title
40,"[{'name': 'RAI Radiotelevisione Italiana SpA', 'address': {'street': 'Viale Mazzini 14', 'city': 'Roma', 'postcode': '00195', 'nuts': ['ITE43'], 'country': 'IT', 'url': 'http://www.rai.it', 'ot': {'nutscode': 'ITI43'}}, 'email': 'portaleacquistirai@bravosolution.com', 'contactPoint': 'Direzione Acquisti e Servizi/Acquisti/BIT', 'contactName': 'dott. Rosario Pappone', 'phone': '+39 0638781', 'mainActivities': ['OTHER', 'GENERAL_PUBLIC_SERVICES'], 'buyerType': 'PUBLIC_BODY', 'id': 'EU_body_cd4420899333c276339617cf35859bc4f68fc07783ceb2cd31784b0aa41b221f'}]",RAI Radiotelevisione Italiana SpA,2011-09-06 12:00:00,"{'netAmount': 11154270, 'currency': 'EUR', 'netAmountEur': 11154270, 'currencyNational': 'EUR', 'netAmountNational': 11154270, 'publicationDate': '2012-02-22'}",2012-02-22,11154270.00,Procedura aperta per l’affidamento dei servizi SOC - Services Operation Center. CIG n. 269882767D.
67,"[{'name': 'Eni SpA', 'address': {'street': 'Via Emilia 1', 'city': 'San Donato Milanese', 'postcode': '20097', 'nuts': ['ITC45'], 'country': 'IT', 'url': 'http://www.eni.com', 'ot': {'nutscode': 'ITC4C'}}, 'email': 'stefano.fasani@eni.com', 'contactPoint': 'APR/UP-D', 'contactName': 'Stefano Fasani', 'phone': '+39 0252063649', 'mainActivities': ['GAS_AND_OIL_EXTRACTION', 'COAL_AND_OTHER_EXTRACTION', 'OTHER'], 'buyerType': 'UTILITIES', 'id': 'EU_body_666f309571a52334fad6defca061443df18fde5881bd23d16b57fade18768ca1'}]",Eni SpA,NaT,NaN,NaN,NaN,Servizi di system integration per la realizzazione del sistema BMS Plus.
87,"[{'name': 'IVASS — Istituto per la vigilanza sulle assicurazioni', 'address': {'street': 'Via del Quirinale 21', 'city': 'Roma', 'postcode': '00187', 'nuts': ['ITE43'], 'country': 'IT', 'url': 'http://www.ivass.it/', 'ot': {'nutscode': 'ITI43'}}, 'email': 'gare.contratti@pec.ivass.it', 'contactPoint': 'Settore gare appalti e contratti', 'contactName': 'Settore Gare Appalti e Contratti', 'phone': '+39 06421331', 'mainActivities': ['OTHER', 'ECONOMIC_AND_FINANCIAL_AFFAIRS'], 'buyerType': 'OTHER', 'id': 'EU_body_17209481c261e97aa91f0c81ae9e0baaa6cd53fcd269b6ef76bff409e5ebb3a5'}]",IVASS — Istituto per la vigilanza sulle assicurazioni,NaT,NaN,NaN,NaN,Fornitura di licenze d'uso software e di relativi servizi di manutenzione nell'ambito del progetto «CAD IVASS» (CIG 6092758EF1).
115,"[{'name': 'RAI Radiotelevisione Italiana SpA', 'address': {'street': 'Viale Mazzini 14', 'city': 'Roma', 'postcode': '00195', 'nuts': ['ITE43'], 'country': 'IT', 'url': 'http://www.rai.it', 'ot': {'nutscode': 'ITI43'}}, 'email': 'portaleacquistirai@bravosolution.com', 'contactPoint': 'Direzione Acquisti e Servizi/Acquisti/BIT', 'contactName': 'dott. Rosario Pappone', 'phone': '+39 0638781', 'mainActivities': ['OTHER', 'GENERAL_PUBLIC_SERVICES'], 'buyerType': 'PUBLIC_BODY', 'id': 'EU_body_cd4420899333c276339617cf35859bc4f68fc07783ceb2cd31784b0aa41b221f'}]",RAI Radiotelevisione Italiana SpA,2015-04-28 12:00:00,"{'netAmount': 761085, 'currency': 'EUR', 'netAmountEur': 761085, 'currencyNational': 'EUR', 'netAmountNational': 761085, 'publicationDate': '2015-12-03'}",2015-12-03,761085.00,Procedura aperta per l'affidamento dei «Servizi di supporto tecnico inerenti la lavorazione dei contenuti per le piattaforme nuovi media» — Gara n. 5955776 — CIG n. 61564016D1.
159,"[{'name': 'LAit - LAzio Innovazione Tecnologica S.p.A.', 'address': {'street': 'VIA ADELAIDE BONO CAIROLI 68', 'city': 'ROMA', 'postcode': '00145', 'nuts': ['ITI43'], 'country': 'IT', 'ot': {'nutscode': 'ITI43'}}, 'email': 'gare@laitspa.it', 'contactPoint': 'Unità di Staff Affari Legali, Gare, Acquisti e Contratti', 'contactName': 'unità organizzativa legale, gare e contratti', 'phone': '+39 0651689871/9830', 'mainActivities': ['OTHER'], 'buyerType': 'PUBLIC_BODY', 'id': 'EU_body_5c0cf21a93a7374eb7da4a05a8e957015cedaca3fa4d11be2cab47858c480a3f'}]",LAit - LAzio Innovazione Tecnologica S.p.A.,NaT,"{'netAmount': 2250000, 'currency

## Somma contratti per anno

In [14]:
for y in range(2014, 2018):
    print(y, it_df[it_df["bidDeadlineYear"] == y].price.sum())

2014 177942007.7
2015 216381794.83
2016 152594117.46
2017 6766574.52
